In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

In [2]:
spark = SparkSession.builder.appName("pyspark-udf").getOrCreate()

spark = (
    SparkSession.builder
                .appName("pyspark-udf")
                .config("spark.sql.repl.eagerEval.enabled", True)
                .config("spark.sql.execution.arrow.pyspark.enabled", True)
                .getOrCreate()
)

In [3]:
df_raw = spark.read.csv(r'..\raw\AmazonSalesData.csv', header=True, inferSchema=True)

print("Importado.")

Importado.


In [4]:
df_raw.show()

+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|             Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+--------------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|              Tuvalu|      Baby Food|      Offline|             H| 5/28/2010|669165933| 6/27/2010|      9925|    255.28|   159.42|    2533654.0| 1582243.5|    951410.5|
|Central America a...|             Grenada|         Cereal|       Online|             C| 8/22/2012|963881480| 9/15/2012|      2804|     205.7|   117.11|     576782.8| 328376.44|   248406.36|
|              Europe|              Russia|Of

In [5]:
df = (
    df_raw.groupBy('Country','Item Type','Sales Channel')
          .agg(
              sum('Total Revenue').alias('Total Revenue'),
              sum('Total Profit').alias('Total Profit'))
)

df.show()

+--------------------+---------------+-------------+------------------+------------+
|             Country|      Item Type|Sales Channel|     Total Revenue|Total Profit|
+--------------------+---------------+-------------+------------------+------------+
|        Sierra Leone|Office Supplies|      Offline|2868580.0500000003|   556131.25|
|        Sierra Leone|     Vegetables|      Offline|          228779.1|    93748.05|
|            Honduras|         Snacks|       Online|          339490.5|    122686.5|
|            Bulgaria|Office Supplies|       Online|        2596374.27|   503358.75|
|             Myanmar|        Clothes|       Online|          648030.4|    435499.2|
|               Kenya|     Vegetables|       Online|         994765.42|   407630.41|
|            Slovenia|      Beverages|      Offline|          221117.0|     72975.6|
|            Djibouti|      Cosmetics|      Offline|         3154398.0|  1254472.05|
|            Malaysia|         Fruits|      Offline|          584

In [6]:
def calculate_profit_margin(revenue,profit):
    
    if revenue < profit:
        return '0.00%'
    
    profit_margin_value = (profit / revenue) * 100

    profit_margin = format(profit_margin_value, '.2f') + '%'

    return profit_margin

In [7]:
calculate_profit_margin_udf = udf(calculate_profit_margin, StringType())

In [11]:
df_udf = (
    df.withColumn("Profit Margin", calculate_profit_margin_udf(col("Total Revenue"), col("Total Profit")))
)


In [14]:

df_udf.show()

Py4JJavaError: An error occurred while calling o108.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 18) (DESKTOP-L6BGI28.mshome.net executor driver): java.io.IOException: Cannot run program "C:\Users\ITMaker\anaconda3\Lib\site-packages\pyspark": CreateProcess error=5, Acceso denegado
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:181)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonEx